In [1]:
from app.loader import add_docs_to_knowledge_base
from app.db import reset_db, create_vectors_index, count_vectors

In [2]:
await reset_db()
await create_vectors_index()

Index idx:vectors created successfully


In [3]:
await add_docs_to_knowledge_base()

Loading documents


100%|██████████| 1/1 [00:04<00:00,  4.14s/it]


Loaded 1 PDF documents

Splitting documents into chunks
Inception.pdf: 127 chunks

Total chunks: 127
Min chunk size: 351 tokens
Max chunk size: 512 tokens
Average chunk size: 496 tokens

Embedding chunks


100%|██████████| 127/127 [00:02<00:00, 45.94it/s]


Adding chunks to vector DB


In [4]:
total_vectors = await count_vectors()
print(f'Total vectors: {total_vectors}')

Total vectors: 127


In [5]:
import json
from app.openai import chat
from app.tools import QueryKnowledgeBaseTool
from app.prompts import MAIN_SYSTEM_PROMPT, RAG_SYSTEM_PROMPT

In [6]:
chat_messages = []
system_message = {'role': 'system', 'content': MAIN_SYSTEM_PROMPT}
openai_tools = [QueryKnowledgeBaseTool.openai_tool_schema()]
user_message = {'role': 'user', 'content': "What's the most resilient parasite?"}
chat_messages.append(user_message)
content, tool_calls = await chat(
    messages=[system_message, *chat_messages],
    tools=openai_tools,
    tool_choice='auto'
)
print(content)
print(tool_calls)

None
[ChatCompletionMessageToolCall(id='call_QCBVgmYUq2XO7BEF8G9qIawU', function=Function(arguments='{"query_input":"What is the most resilient parasite?"}', name='QueryKnowledgeBaseTool'), type='function')]


In [7]:
tool_call = tool_calls[0]
kb_tool = QueryKnowledgeBaseTool(**json.loads(tool_call.function.arguments))
kb_result = await kb_tool()
chat_messages.extend([
    {'role': 'assistant', 'content': content, 'tool_calls': tool_calls},
    {'role': 'tool', 'tool_call_id': tool_call.id, 'content': kb_result}
])

In [9]:
rag_system_message = {'role': 'system', 'content': RAG_SYSTEM_PROMPT}
content, tool_calls = await chat(messages=[rag_system_message, *chat_messages])
print(content)

The most resilient parasite is described as an idea. Once an idea takes hold in the brain, it becomes almost impossible to eradicate. As Cobb explains, "A person can cover it up, ignore it - but it stays there." This highlights the tenacity of ideas compared to other forms of information, which can be forgotten.
